In [1]:
from Gprocess import roach as rch
import matplotlib.pyplot as plt
import tqdm as tqdm
from time import time

from gem.utils import graph_util
from gem.evaluation import visualize_embedding as viz
from gem.evaluation import evaluate_graph_reconstruction as gr

from gem.embedding.hope import HOPE
from gem.embedding.lap import LaplacianEigenmaps
from gem.embedding.lle import LocallyLinearEmbedding
from argparse import ArgumentParser

In [2]:
l = []
with open('subgraph_wn18.edgelist') as data:
    for line in data:
        l.append(line)
triple = rch.split_raw_triple(l, ' ')
triple

[['23354', '29432\n'],
 ['23354', '24094\n'],
 ['23354', '32401\n'],
 ['23354', '37497\n'],
 ['23354', '1125\n'],
 ['23354', '9006\n'],
 ['23354', '3722\n'],
 ['23354', '26349\n'],
 ['23354', '37044\n'],
 ['23354', '7098\n'],
 ['23354', '25345\n'],
 ['23354', '23216\n'],
 ['23354', '37077\n'],
 ['23354', '29516\n'],
 ['23354', '8182\n'],
 ['23354', '20771\n'],
 ['23354', '23715\n'],
 ['23354', '12273\n'],
 ['23354', '6766\n'],
 ['23354', '32128\n'],
 ['23354', '10756\n'],
 ['23354', '31761\n'],
 ['23354', '8528\n'],
 ['23354', '37593\n'],
 ['23354', '37113\n'],
 ['23354', '22618\n'],
 ['23354', '25624\n'],
 ['23354', '35924\n'],
 ['23354', '29582\n'],
 ['23354', '34193\n'],
 ['23354', '13667\n'],
 ['23354', '19569\n'],
 ['23354', '23774\n'],
 ['23354', '3832\n'],
 ['23354', '17524\n'],
 ['23354', '30995\n'],
 ['23354', '3163\n'],
 ['23354', '38269\n'],
 ['23354', '35487\n'],
 ['23354', '13735\n'],
 ['23354', '6114\n'],
 ['23354', '2988\n'],
 ['23354', '24549\n'],
 ['23354', '12583\n'],

In [3]:
len(triple)

520

In [4]:
for edge in triple:
    edge[0] = int(edge[0])
    edge[1] = int(edge[1].strip())
triple

[[23354, 29432],
 [23354, 24094],
 [23354, 32401],
 [23354, 37497],
 [23354, 1125],
 [23354, 9006],
 [23354, 3722],
 [23354, 26349],
 [23354, 37044],
 [23354, 7098],
 [23354, 25345],
 [23354, 23216],
 [23354, 37077],
 [23354, 29516],
 [23354, 8182],
 [23354, 20771],
 [23354, 23715],
 [23354, 12273],
 [23354, 6766],
 [23354, 32128],
 [23354, 10756],
 [23354, 31761],
 [23354, 8528],
 [23354, 37593],
 [23354, 37113],
 [23354, 22618],
 [23354, 25624],
 [23354, 35924],
 [23354, 29582],
 [23354, 34193],
 [23354, 13667],
 [23354, 19569],
 [23354, 23774],
 [23354, 3832],
 [23354, 17524],
 [23354, 30995],
 [23354, 3163],
 [23354, 38269],
 [23354, 35487],
 [23354, 13735],
 [23354, 6114],
 [23354, 2988],
 [23354, 24549],
 [23354, 12583],
 [23354, 24449],
 [23354, 37571],
 [23354, 25618],
 [23354, 23163],
 [23354, 39100],
 [23354, 21238],
 [23354, 37992],
 [23354, 10655],
 [23354, 22844],
 [23354, 33072],
 [23354, 10292],
 [23354, 15726],
 [23354, 34035],
 [23354, 34934],
 [23354, 34879],
 [23354,

In [5]:
len(triple)

520

In [6]:
key = rch.nodemap(triple)
key
keyinv = rch.mapback(key)
len(key)

521

In [7]:
key

{23354: 0,
 29432: 1,
 24094: 2,
 32401: 3,
 37497: 4,
 1125: 5,
 9006: 6,
 3722: 7,
 26349: 8,
 37044: 9,
 7098: 10,
 25345: 11,
 23216: 12,
 37077: 13,
 29516: 14,
 8182: 15,
 20771: 16,
 23715: 17,
 12273: 18,
 6766: 19,
 32128: 20,
 10756: 21,
 31761: 22,
 8528: 23,
 37593: 24,
 37113: 25,
 22618: 26,
 25624: 27,
 35924: 28,
 29582: 29,
 34193: 30,
 13667: 31,
 19569: 32,
 23774: 33,
 3832: 34,
 17524: 35,
 30995: 36,
 3163: 37,
 38269: 38,
 35487: 39,
 13735: 40,
 6114: 41,
 2988: 42,
 24549: 43,
 12583: 44,
 24449: 45,
 37571: 46,
 25618: 47,
 23163: 48,
 39100: 49,
 21238: 50,
 37992: 51,
 10655: 52,
 22844: 53,
 33072: 54,
 10292: 55,
 15726: 56,
 34035: 57,
 34934: 58,
 34879: 59,
 15067: 60,
 27364: 61,
 15476: 62,
 5284: 63,
 31721: 64,
 25565: 65,
 21622: 66,
 16258: 67,
 37397: 68,
 37197: 69,
 2587: 70,
 34281: 71,
 3012: 72,
 11106: 73,
 26373: 74,
 24272: 75,
 29727: 76,
 32300: 77,
 13132: 78,
 82: 79,
 24699: 80,
 15712: 81,
 3461: 82,
 40643: 83,
 40061: 84,
 2462: 8

In [8]:
ntriple = rch.preprocess(triple, key)
ntriple

[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [0, 10],
 [0, 11],
 [0, 12],
 [0, 13],
 [0, 14],
 [0, 15],
 [0, 16],
 [0, 17],
 [0, 18],
 [0, 19],
 [0, 20],
 [0, 21],
 [0, 22],
 [0, 23],
 [0, 24],
 [0, 25],
 [0, 26],
 [0, 27],
 [0, 28],
 [0, 29],
 [0, 30],
 [0, 31],
 [0, 32],
 [0, 33],
 [0, 34],
 [0, 35],
 [0, 36],
 [0, 37],
 [0, 38],
 [0, 39],
 [0, 40],
 [0, 41],
 [0, 42],
 [0, 43],
 [0, 44],
 [0, 45],
 [0, 46],
 [0, 47],
 [0, 48],
 [0, 49],
 [0, 50],
 [0, 51],
 [0, 52],
 [0, 53],
 [0, 54],
 [0, 55],
 [0, 56],
 [0, 57],
 [0, 58],
 [0, 59],
 [0, 60],
 [0, 61],
 [0, 62],
 [0, 63],
 [0, 64],
 [0, 65],
 [0, 66],
 [0, 67],
 [0, 68],
 [0, 69],
 [0, 70],
 [0, 71],
 [0, 72],
 [0, 73],
 [0, 74],
 [0, 75],
 [0, 76],
 [0, 77],
 [0, 78],
 [0, 79],
 [0, 80],
 [0, 81],
 [0, 82],
 [0, 83],
 [0, 84],
 [0, 85],
 [0, 86],
 [0, 87],
 [0, 88],
 [0, 89],
 [0, 90],
 [0, 91],
 [0, 92],
 [0, 93],
 [0, 94],
 [0, 95],
 [0, 96],
 [0, 97],
 [0, 98],
 [0, 99],
 [0, 100],
 [0, 101

In [9]:
degree = rch.getdegreedist(ntriple)
degree

{0: 520}

In [10]:
with open('sg_wn18processed.edgelist', 'w') as s:
    for item in ntriple:
        s.write('{} {}\n'.format(item[0], item[1]))
    print('File written successfully')
                 
s.close()

File written successfully


In [11]:
edge_f = "sg_wn18processed.edgelist"
isDirected = True
G = graph_util.loadGraphFromEdgeListTxt(edge_f, directed=isDirected)
G = G.to_directed()

In [12]:
nodes = list(G.nodes())

In [13]:
with open('sg_wn18processednodes.edgelist', 'w') as s:
    for item in nodes:
        s.write('{}\n'.format(item))
    print('File written successfully')
                 
s.close()

File written successfully


In [14]:
embedding = HOPE(d = 2, beta = 0.01)
print('Num nodes: %d, num edges: %d' % (G.number_of_nodes(), G.number_of_edges()))
t1 = time()
        # Learn embedding - accepts a networkx graph or file with edge list
Y = embedding.learn_embedding(graph=G, is_weighted=True, no_python=True)
print(embedding.get_method_name()+':\n\tTraining time: %f' % (time() - t1))
        # Evaluate on graph reconstruction
MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G, embedding, Y, None)
        # ---------------------------------------------------------------------------------
print(("\tMAP: {} \t preccision curve: {}\n\n\n\n"+'-'*100).format(MAP, prec_curv[:5]))

Num nodes: 521, num edges: 520
SVD error (low rank): 0.000000
hope_gsvd:
	Training time: 0.052082
	MAP: 0.0019193857965451055 	 preccision curve: [1.0, 1.0, 1.0, 1.0, 1.0]



----------------------------------------------------------------------------------------------------
